## DC Inbox

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)

## Importing .csvs

In [3]:
csv_file_list = ['12_2020.csv', '01_2021.csv', '02_2021 (1).csv', '03_2021 (1).csv', '04_2021.csv', '05_2021.csv']

list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

df = pd.concat(list_of_dataframes)

In [4]:
df.head()

,Date,Subject,Message,bioguide_id,From
0,12/31/20 18:54,10 Wins for North Carolina,"A message from Senator Thom Tillis \n￼\nEmail Me\nThom’s Results\nMeet Thom\nHow Can I Help?\nFriends, \nLast week, the Senate passed the bipartisan omnibus and COVID-19 relief bill, which I suppo...",T000476,NaN
1,12/31/20 17:51,Advocating For Nevadans Is My Top Priority – Read My 2020 Annual Report,"￼\n \nFellow Nevadans, 2020 has been an unbelievably challenging year. In the midst of a once-in-a-lifetime pandemic, people in the Silver State have quarantined, socially distanced, and masked ...",C001113,NaN
2,12/31/20 17:33,District Update: 2020 Year in Review,"COVID-19 Update | December 31, 2020\nView Email in Browser \n￼ \n\nDear Friend,\n\nI hope that everyone across Arkansas had an enjoyable holiday season spending time with friends and family.\n\n20...",H001072,NaN
3,12/31/20 17:16,A Year in Review,"\n￼\n￼\n\nDear friends and neighbors, \n\nIt’s been a busy two years fighting for the issues that matter most to Central Florida’s families. For many in Florida and across the country, 2020 has be...",S001200,NaN
4,12/31/20 17:14,"Happy New Year, Marianas!","\n\nEmail not displaying correctly? Click here.\n\n \n \n \n\nThis week in Congress\nDecember 31, 2020\n\n\t•\tCOVID vaccinations begin\n\t•\tMore for Marianas in FY21 appropriation:\n\t•\tAnother...",S001177,NaN


In [5]:
df.shape

(5800, 5)

In [6]:
df = df.dropna(how='all')

In [7]:
df.shape

(5783, 5)

In [7]:
# df[df.Message.isna()]

# Setting up the scraper

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import lxml
driver = webdriver.Chrome()

In [13]:
driver.get("https://www.congress.gov/help/field-values/member-bioguide-ids")

In [14]:
table_rows = driver.find_elements_by_tag_name('tbody')

In [15]:
for row in table_rows:
    row_list = row.find_elements_by_tag_name('tr')

In [16]:
row_list[1].find_element_by_tag_name('td').text.strip()

'Abdnor, James (Republican - South Dakota)'

In [27]:
congress_IDs = []
for row in row_list[1:]:
    try:
        member_name = row.find_elements_by_tag_name('td')[0].text.strip()
        member_id = row.find_elements_by_tag_name('td')[1].text.strip()
    except:
        pass
        results = {
            'member_name': member_name,
            'member_id': member_id
        }
        congress_IDs.append(results)
congress_IDs

[{'member_name': 'Abdnor, James (Republican - South Dakota)',
  'member_id': 'A000009'},
 {'member_name': 'Abercrombie, Neil (Democratic - Hawaii)',
  'member_id': 'A000014'},
 {'member_name': 'Abourezk, James (Democratic - South Dakota)',
  'member_id': 'A000017'},
 {'member_name': 'Abraham, Ralph Lee (Republican - Louisiana)',
  'member_id': 'A000374'},
 {'member_name': 'Abraham, Spencer (Republican - Michigan)',
  'member_id': 'A000355'},
 {'member_name': 'Abzug, Bella S. (Democratic - New York)',
  'member_id': 'A000018'},
 {'member_name': 'Acevedo-Vila, Anibal (Democratic - Puerto Rico)',
  'member_id': 'A000359'},
 {'member_name': 'Ackerman, Gary L. (Democratic - New York)',
  'member_id': 'A000022'},
 {'member_name': 'Adams, Alma S. (Democratic - North Carolina)',
  'member_id': 'A000370'},
 {'member_name': 'Adams, Brock (Democratic - Washington)',
  'member_id': 'A000031'},
 {'member_name': 'Adams, Sandy (Republican - Florida)',
  'member_id': 'A000366'},
 {'member_name': 'Adda

In [28]:
df1 = pd.DataFrame(congress_IDs)

In [8]:
# df1.to_csv('representatives.csv', index=False)

In [8]:
df1 = pd.read_csv('representatives.csv')

# Cleaning Scraped Data

In [9]:
df1['name'] = df1.member_name.str.extract("^(.+)\s\(.*")

In [10]:
df1['party'] = df1.member_name.str.extract("\s\((\w+)\s-\s\w+")

In [11]:
df1['state'] = df1.member_name.str.extract("-\s(.+)\)")

# Merging Dataframes

In [12]:
merged = df.merge(df1, how='left', left_on='bioguide_id', right_on='member_id')

In [13]:
merged.shape

(5783, 10)

In [14]:
merged = merged.drop(columns=['member_name'])

In [40]:
merged.party.value_counts()

Republican     3172
Democratic     2590
Independent      19
Name: party, dtype: int64

In [41]:
merged.to_csv('newsletters1220-0521.csv', index=False)

In [172]:
merged.to_csv('newsletters1220-0521indexed.csv')

In [15]:
import re

In [16]:
# MergedVacMention = merged[merged.Message.str.contains("vaccin", flags=re.IGNORECASE, regex=True, na=False)]

In [17]:
MergedVacMention = merged[(merged.Message.str.contains("vaccine", flags=re.IGNORECASE, regex=True, na=False)) | (merged.Message.str.contains("vaccination", flags=re.IGNORECASE, regex=True, na=False))]

In [46]:
MergedVacMention.shape

(1720, 9)

In [365]:
# DecOnly = MergedVacMention[MergedVacMention.Date.str.contains("(12/\d\d/\d\d)")]

In [364]:
# DecOnly.party.value_counts()

In [358]:
# MergedVacMention[MergedVacMention.party == "Republican"].name.value_counts()

### Taking a sample to be labeled

I did not originally take a 'random_state' sample and neither did I save the index with the sample as I was unaware of the problems I would face later on while merging the labeled df with the main df. Hence, the removal of duplicates manually after I joined both on 'Date'.

In [113]:
# sampledf = MergedVacMention.sample(500)

In [117]:
# sampledf.party.value_counts()

In [114]:
# sampledf.to_csv('sampleVacMention.csv', index=False)

# Importing Labeled Data and Buidling the Classifier

In [18]:
labeled = pd.read_csv("/Users/khanfazil/Desktop/github_repositories/foundations_homework/algos_project/sampleVacMentionLabeled350.csv")

In [19]:
labeled = labeled.dropna(subset = ['VacHesitant/AntiVac'])

In [20]:
labeled = labeled.rename(columns={'VacHesitant/AntiVac': 'label'})

In [21]:
labeled['label'] = labeled.label.replace({
    'Yes': 1,
    'No': 0
})

In [45]:
labeled.party.value_counts()

Democratic     174
Republican     174
Independent      2
Name: party, dtype: int64

In [22]:
df = labeled.merge(MergedVacMention, on='Date', how='right')

In [23]:
df.shape

(1724, 18)

### Deleting duplicates

In [24]:
df[(df.name_x != df.name_y) & df.label.notna()]

,Date,Subject_x,Message_x,bioguide_id_x,From_x,member_id_x,name_x,party_x,state_x,label,Subject_y,Message_y,bioguide_id_y,From_y,member_id_y,name_y,party_y,state_y
149,12/15/20 13:13,REMINDER: Open Enrollment for 2021 Ends TODAY,"News from Representative Bobby Scott\n\n \n\nAbout Bobby\nContact\nServices\nLegislation\n3rd District\n \n December 15, 2020 \nREMINDER: Open Enrollment for 2021 Ends TODAY\nDear Friend, \nToday ...",S000185,NaN,S000185,"Scott, Robert C. ""Bobby""",Democratic,Virginia,0.0,A holiday message from Rep. Charlie Crist,"￼ \n \n\nA light at the end of the tunnel\n\nDear neighbor, \nThe gift we've all had on our lists this holiday season is finally on its way: a safe, effective COVID-19 vaccine. Healthcare workers...",C001111,NaN,C001111,"Crist, Charlie",Democratic,Florida
376,1/29/21 17:45,Serving You,"Dear Friend,\n\nThank you for subscribing to my e-newsletter, where you will get the latest updates on my work for you in Congress. My team and I are here to serve the 39th District of California ...",K000397,NaN,K000397,"Kim, Young",Republican,California,0.0,From the Desk of Congresswoman Diana Harshbarger,"\n￼\n￼\n￼\n\nDear fellow East Tennesseans,\n\nThis week I met with folks here in East Tennessee. I also received news of my committee assignments for the 117th Congress. I’m excited to share this ...",H001086,NaN,H001086,"Harshbarger, Diana",Republican,Tennessee
845,2/27/21 10:03,Weekly Update: COVID Relief Hijacked by Partisan Politics,"Weekly Update | February 27, 2021\n \n\nCOVID Relief Hijacked by Partisan Politics\n\nToday the Democrats’ partisan COVID relief bill passed the House. This action followed their repeated rejectio...",M001159,NaN,M001159,"McMorris Rodgers, Cathy",Republican,Washington,0.0,"Honoring the Life of Luke Letlow, Fighting to Reopen Schools, and More","￼\nWebsite | About | Contact Me | Serving You | Media Center | Our District \n\nFebruary 27, 2021\n \nTHE SCALISE CAPITOL REPORT\n \nHonoring the Life of Congressman-Elect Luke Letl...",S001176,NaN,S001176,"Scalise, Steve",Republican,Louisiana
848,2/27/21 10:03,"Honoring the Life of Luke Letlow, Fighting to Reopen Schools, and More","Website | About | Contact Me | Serving You | Media Center | Our District \n\nFebruary 27, 2021\n \nTHE SCALISE CAPITOL REPORT\n \nHonoring the Life of Congressman-Elect Luke Letlow\...",S001176,NaN,S001176,"Scalise, Steve",Republican,Louisiana,0.0,Weekly Update: COVID Relief Hijacked by Partisan Politics,"\n\nWeekly Update | February 27, 2021\n \n\nCOVID Relief Hijacked by Partisan Politics\n\nToday the Democrats’ partisan COVID relief bill passed the House. This action followed their repeated reje...",M001159,NaN,M001159,"McMorris Rodgers, Cathy",Republican,Washington
1053,4/16/21 15:35,Brett's Bulletin: Updates for KY-02,"April 16, 2021 \n \n \n \n\nDear Friends,\nLast week I had the opportunity to tour vaccine clinics in the Second District and receive updates from hospital leaders on how health care providers hav...",G000558,NaN,G000558,"Guthrie, Brett",Republican,Kentucky,0.0,COVID-19 Updates,"￼ \n \n\nDear Neighbors,\nFunding from the American Rescue Plan is on the way to states to help identify emerging viral threats early. New Jersey will receive nearly $5.8 million. I helped author...",P000034,NaN,P000034,"Pallone, Frank, Jr.",Democratic,New Jersey
1253,4/30/21 16:25,I'm hosting Mass Vaccination Week,"About\nContact\nIssues\nMedia Center\nServices\nDear Lindsey , \nI have exciting news to share with you. Starting tomorrow, Saturday, May 1st thru Saturday, May 8th, I’m spearheading Michigan’s 1...",L000581,NaN,L000581,"Lawrence, Brenda L.",Democratic,Michigan,0.0,THURSDAY: Another Opportunity to Connect with Employers,"April 30, 2021\nShare on Facebook | Share on Twitter | Permalink \n￼\nTHURSDAY: Another Opportunity to Connect with Potential Employers\n￼\nAs vaccination rates are going up, our country is moving...",S001187,NaN,S001187,"Stivers, Steve",Republican,Ohio
1268,4/30/21 18:10,McKinle

In [25]:
df = df.drop([149, 376, 845, 848, 1053, 1253, 1268, 1274, 1470, 1473])

In [26]:
df = df.drop(columns=(['Subject_x', 'Message_x', 'bioguide_id_x', 'From_x',
                  'member_id_x', 'name_x', 'party_x', 'state_x', 'From_y']))
df = df.rename(columns=({
    'Message_y': 'Message',
    'bioguide_id_y': 'bioguide_id',
    'member_id_y': 'member_id',
    'name_y': 'name',
    'party_y': 'party',
    'state_y': 'state'
}))

In [27]:
df.shape

(1714, 9)

In [44]:
df.name.value_counts().head(10)

McMorris Rodgers, Cathy    42
Rice, Tom                  31
Fletcher, Lizzie           27
Bilirakis, Gus M.          27
Manchin, Joseph            24
Cooper, Jim                23
Dingell, Debbie            22
Espaillat, Adriano         21
Kuster, Ann M.             21
Pallone, Frank, Jr.        21
Name: name, dtype: int64

## Vectorizer

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(max_features=1000, min_df=20, stop_words = 'english', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(labeled.Message)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

# vectorizer = TfidfVectorizer(min_df=20, max_df=0.7, ngram_range=(1, 3))
# vectors = vectorizer.fit_transform(df.text)
# words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
# # words_df.index = df.text
# words_df.head()

,00,000,10,100,11,12,13,14,15,16,17,18,19,19 pandemic,19 relief,19 testing,19 vaccination,19 vaccine,19 vaccines,20,200,2019,202,202 225,2020,2021,20515,20515 phone,20515 phone 202,20515 unsubscribe,21,225,25,30,300,400,500,600,65,800,833,able,access,act,action,actions,addition,additional,additionally,address,...,video,view,view email,view email browser,violence,virginia,virtual,virus,visit,visit website,visited,visiting,vital,vote,voted,vulnerable,walk,want,washington,washington dc,washington dc 20515,washington dc office,watch,water,way,ways,wear,website,website unsubscribe,wednesday,week,weekly,weeks,welcome,west,white,wish,women,work,worked,workers,workforce,working,world,www,year,years,yesterday,york,young
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.151734,0.0,0.051672,0.0,0.0,0.098102,0.000000,0.000000,0.000000,0.0,0.028246,0.000000,0.038911,0.091916,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.037613,0.031134,0.0,0.0,0.0,0.040461,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.061671,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.091042,0.078571,0.000000,0.000000,0.000000,0.000000,0.038036,0.0,0.0,0.000000,0.000000,0.000000,0.097603,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.026745,0.0,0.035250,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,0.084293,0.000000,0.0,0.0,0.000000,0.0,0.0,0.038969,0.0,0.000000,0.0,0.0,0.058789,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055082,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.055973,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.107235,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.108013,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.087735,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.048081,0.0,0.063372,0.000000,0.000000,0.089016,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.090797,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.039439,0.044038,0.000000,0.042780,0.039560,0.052324,0.052324,0.0,0.0,0.043472,0.000000,0.074214,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.466264,0.075533,0.0,0.043054,0.067407,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.063559,0.073138,0.041577,0.057178,0.000000,0.000000,0.000000,0.0,0.0,0.042373,0.000000,0.131562,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.121619,0.000000,0.032259,0.0,0.0,0.027588,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.015493,0.017300,0.000000,0.033612,0.015541,0.020555,0.020555,0.0,0.0,0.017078,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.023916,0.000000,0.085389,0.0,0.0,0.0,0.022193,0.0,0.046233,...,0.032718,0.017300,0.020256,0.020256,0.0,0.000000,0.000000,0.0,0.016914,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.07615,0.0,0.024969,0.028732,0.016333,0.022462,0.069349,0.000000,0.020864,0.0,0.0,0.016646,0.000000,0.000000,0.107075,0.0,0.0,0.0,0.0,0.0,0.0,0.028823,0.073350,0.0,0.000000,0.208679,0.017997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.049463,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.060751,0.0,0.0,0.078546,0.0,0.000000,0.0,0.0,0.039498,0.064574,0.061734,0.072047,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.068237,0.064200,0.0,0.0,0.073103,0.0,0.0,0.0,0.0,0.000000,0.045432,0.000000,0.0,0.0,0.0,0.048871,0.0,0.000000,...,0.000000,0.038096,0.044604,0.044604,0.0,0.000000,0.000000,0.0,0.037245,0.058311,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.145139,0.045943,0.0,0.0,0.073311,0.071536,0.000000,0.058946,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

# English stemmer from pyStemmer
stemmer = Stemmer.Stemmer('en')

analyzer = TfidfVectorizer().build_analyzer()

# Override TfidfVectorizer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: stemmer.stemWords(analyzer(doc))

# Create a new StemmedTfidfVectorizer
vectorizer = StemmedTfidfVectorizer(max_features=2000, stop_words = 'english', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(labeled.Message)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,00,00 pm,000,10,100,100 day,101,11,117th,117th congress,12,13,14,15,15th,16,17,18,19,19 case,19 pandem,19 relief,19 relief packag,19 resourc,19 test,19 vaccin,19 vaccine distribut,20,200,2019,202,202 224,202 225,202 226,2020,2021,2021 dear,20510,20515,20515 202,20515 202 225,20515 phone,20515 phone 202,20515 unsubscrib,20515 unsubscribe gmail,21,215,22,224,225,...,website http,website unsubscrib,website unsubscribe contact,wednesday,week,week join,week sen,weekend,welcom,west,west virginia,west virginia depart,west virginian,whip,white,white hous,wichita,wide,will,william,win,window,winner,winter,wish,wit,women,won,wonder,word,work,work behalf,work congress,worker,workforc,world,write,wrong,wrote,wv,www,xl,year,years ago,years old,yes,yesterday,york,young,youtub
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.095767,0.000000,0.0,0.032613,0.043375,0.0,0.0,0.054819,0.0,0.000000,0.000000,0.0,0.017827,0.0414,0.000000,0.0,0.024559,0.058013,0.0,0.046253,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.040019,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.059317,0.0,0.103121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.135655,0.0,0.0,0.000000,0.0,0.0,0.0,0.014744,0.0,0.0,0.022016,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.068538,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.031686,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.042321,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.0,0.000000,0.044787,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.068690,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089132,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.034147,0.0,0.0,0.050990,0.000000,0.072378,0.0,0.111143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.076729,0.076071,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.033328,0.0000,0.037215,0.0,0.000000,0.036152,0.0,0.000000,0.033430,0.0,0.0,0.044217,0.044217,0.0,0.0,0.0,0.0,0.0,0.000000,0.036736,...,0.000000,0.000000,0.000000,0.109091,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071946,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0945,0.0,0.0,0.000000,0.025066,0.0,0.0,0.0,0.021437,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.012039,0.0000,0.013443,0.0,0.000000,0.026117,0.0,0.000000,0.012076,0.0,0.0,0.015972,0.015972,0.0,0.0,0.0,0.0,0.0,0.000000,0.013270,...,0.029291,0.000000,0.000000,0.000000,0.080112,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.022396,0.0,0.0,0.0,0.059737,0.0,0.0,0.000000,0.162148,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.03646,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.044781,0.0,0.0,0.057898,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.051554,0.0,0.045506,0.053108,0.0,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.052731,0.054701,0.000000,0.041838,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.031057,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Training the Model

In [30]:
X = words_df
y = labeled.label

In [31]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

## With Linear SVC

In [33]:
%%time
# clf = LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)
# clf = RandomForestClassifier(n_estimators=50)
clf = LinearSVC()
# clf = MultinomialNB()
clf.fit(X, y)

CPU times: user 23 ms, sys: 2 ms, total: 25 ms
Wall time: 25.7 ms


LinearSVC()

## Predicting Unknown DF

In [34]:
unknown_df = df[df.label.isna()]

In [35]:
unknown_df = unknown_df.drop(columns=['label'])

In [36]:
matrix = vectorizer.transform(unknown_df.Message)

In [37]:
unknown_df['prediction'] = clf.predict(matrix)

In [38]:
unknown_df.prediction.value_counts()

0    1364
Name: prediction, dtype: int64

### Rough

In [326]:
# feature_names = X.columns
# coefficients = clf.coef_[0]

# pd.DataFrame({
#     'feature': feature_names,
#     'coefficient': coefficients
# }).sort_values(by='coefficient', ascending=False)

In [327]:
# from sklearn.metrics import confusion_matrix

In [329]:
# import eli5

# eli5.show_weights(clf, top=(10, 10), feature_names=vectorizer.get_feature_names())

In [153]:
# feature_names

In [310]:
# y_true = y_test
# y_pred = clf.predict(X_test)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['not antivac', 'antivac'])
# pd.DataFrame(matrix,
#      columns='Predicted ' + label_names,
#      index='Is ' + label_names)

In [124]:
# MergedVacMention['prediction'] = clf.predict(feature_names)
# MergedVacMention.head()

## This Notebook ends here. Any code below this is from a previous assignment.

In [330]:
# labeled = pd.read_csv("covid_mention.csv").rename(columns={'covid_mention': 'label'})
# labeled.head()

In [331]:
# df2 = labeled.merge(merged, on='Date', how='right')

In [359]:
# df2 = df2.drop(columns=['bioguide_id_x', 'bioguide_id_y'])

In [360]:
# df2.label = df2.label.str.replace("no", "No")

In [332]:
# df2.label.value_counts()

In [333]:
# df2['covid_mention'] = (df2.label.dropna() == "Yes").astype(int)

In [334]:
# labeled_df = df2.dropna()

In [335]:
# train_df = pd.DataFrame({
#     'covid_mention': labeled_df.covid_mention,
#     'COVID-19': labeled_df.Message.str.contains("COVID-19", na=False).astype(int),
#     'coronavirus': labeled_df.Message.str.contains("coronavirus", na=False).astype(int),
#     'Coronavirus': labeled_df.Message.str.contains("Coronavirus", na=False).astype(int),
#     'pandemic': labeled_df.Message.str.contains("pandemic", na=False).astype(int),
#     'virus': labeled_df.Message.str.contains("virus", na=False).astype(int)
# })
# train_df

In [336]:
# X = train_df.drop(columns='covid_mention')
# y = train_df.covid_mention

In [337]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y)

# Logistic

In [338]:
# from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)
# clf.fit(X, y)

In [339]:
# clf.fit(X_train, y_train)

In [340]:
# clf.score(X_test, y_test)

In [341]:
# feature_names = X.columns
# coefficients = clf.coef_[0]

# pd.DataFrame({
#     'feature': feature_names,
#     'coefficient': coefficients
# }).sort_values(by='coefficient', ascending=False)

In [342]:
# from sklearn.metrics import confusion_matrix

# y_true = y
# y_pred = clf.predict(X)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['covid_not_mentioned', 'covid_mention'])
# pd.DataFrame(matrix,
#      columns='Predicted ' + label_names,
#      index='Is ' + label_names)

In [343]:
# features = pd.DataFrame({
#     'COVID-19': merged.Message.str.contains("COVID-19", na=False).astype(int),
#     'coronavirus': merged.Message.str.contains("coronavirus", na=False).astype(int),
#     'Coronavirus': merged.Message.str.contains("Coronavirus", na=False).astype(int),
#     'pandemic': merged.Message.str.contains("pandemic", na=False).astype(int),
#     'virus': merged.Message.str.contains("virus", na=False).astype(int)
# })

In [344]:
# merged['prediction'] = clf.predict(features)
# merged.head()

In [345]:
# len(merged[merged.prediction == 1])

# DecisionTree

In [346]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [347]:
# from sklearn.tree import DecisionTreeClassifier

# clf = DecisionTreeClassifier()
# clf.fit(X_train, y_train)

In [348]:
# clf.score(X_test, y_test)

In [349]:
# from sklearn.metrics import confusion_matrix

# y_true = y
# y_pred = clf.predict(X)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['covid_not_mentioned', 'covid_mention'])
# pd.DataFrame(matrix,
#      columns='Predicted ' + label_names,
#      index='Is ' + label_names)

# Random Forest

In [350]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [351]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=100)
# clf.fit(X_train, y_train)

In [352]:
# clf.score(X_test, y_test)

In [353]:
# from sklearn.metrics import confusion_matrix

# y_true = y
# y_pred = clf.predict(X)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['covid_not_mentioned', 'covid_mention'])
# pd.DataFrame(matrix,
#      columns='Predicted ' + label_names,
#      index='Is ' + label_names)

# Linear SVC

In [354]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [355]:
# from sklearn.svm import LinearSVC

# clf = LinearSVC(class_weight='balanced')
# clf.fit(X_train, y_train)

In [356]:
# clf.score(X_test, y_test)

In [357]:
# from sklearn.metrics import confusion_matrix

# y_true = y
# y_pred = clf.predict(X)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['covid_not_mentioned', 'covid_mention'])
# pd.DataFrame(matrix,
#      columns='Predicted ' + label_names,
#      index='Is ' + label_names)